# Predicting Teacher Turnover

__Meaghan Ross__

Flatiron School Capstone

### Business Understanding

### Data Understanding

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

### Data Preparation

In [2]:
ls data/

SPR_SY1415_School_Metric_Scores_20160208.xlsx
SPR_SY1516_School_Metric_Scores_20170203.xlsx
SPR_SY1617_School_Metric_Scores_20180206.xlsx
SPR_SY1718_School_Metric_Scores_20190129.xlsx
SPR_SY1819_School_Metric_Scores_20200127.xlsx
employee_information_April2015.csv
employee_information_April2016.csv
employee_information_April2017.csv
employee_information_April2018.csv
employee_information_April2019.csv
employee_information_April2020.csv
employee_information_April2021.csv


In [3]:
df_2015=pd.read_csv('data/employee_information_April2015.csv')
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19138 entries, 0 to 19137
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      19137 non-null  object
 1   FIRST_NAME                     19138 non-null  object
 2   PAY_RATE_TYPE                  19138 non-null  object
 3   PAY_RATE                       19138 non-null  int64 
 4   TITLE_DESCRIPTION              19138 non-null  object
 5   HOME_ORGANIZATION              19138 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  19138 non-null  object
 7   ORGANIZATION_LEVEL             19138 non-null  object
 8   TYPE_OF_REPRESENTATION         19138 non-null  object
 9   GENDER                         19138 non-null  object
 10  RUN_DATE                       19138 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.6+ MB


In [4]:
df_2016=pd.read_csv('data/employee_information_April2016.csv')
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      17561 non-null  object
 1   FIRST_NAME                     17562 non-null  object
 2   PAY_RATE_TYPE                  17562 non-null  object
 3   PAY_RATE                       17562 non-null  int64 
 4   TITLE_DESCRIPTION              17562 non-null  object
 5   HOME_ORGANIZATION              17562 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  17562 non-null  object
 7   ORGANIZATION_LEVEL             17562 non-null  object
 8   TYPE_OF_REPRESENTATION         17561 non-null  object
 9   GENDER                         17562 non-null  object
 10  RUN_DATE                       17562 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.5+ MB


In [5]:
df_2017 = pd.read_csv('data/employee_information_April2017.csv')
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18353 entries, 0 to 18352
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      18352 non-null  object
 1   FIRST_NAME                     18353 non-null  object
 2   PAY_RATE_TYPE                  18353 non-null  object
 3   PAY_RATE                       18353 non-null  int64 
 4   TITLE_DESCRIPTION              18353 non-null  object
 5   HOME_ORGANIZATION              18353 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  18353 non-null  object
 7   ORGANIZATION_LEVEL             18353 non-null  object
 8   TYPE_OF_REPRESENTATION         18343 non-null  object
 9   GENDER                         18353 non-null  object
 10  RUN_DATE                       18353 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.5+ MB


In [6]:
df_2018 = pd.read_csv('data/employee_information_April2018.csv')
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19181 entries, 0 to 19180
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      19180 non-null  object
 1   FIRST_NAME                     19181 non-null  object
 2   PAY_RATE_TYPE                  19181 non-null  object
 3   PAY_RATE                       19181 non-null  int64 
 4   TITLE_DESCRIPTION              19181 non-null  object
 5   HOME_ORGANIZATION              19181 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  19181 non-null  object
 7   ORGANIZATION_LEVEL             19181 non-null  object
 8   TYPE_OF_REPRESENTATION         18864 non-null  object
 9   GENDER                         19181 non-null  object
 10  RUN_DATE                       19181 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.6+ MB


In [7]:
df_2019 = pd.read_csv('data/employee_information_April2019.csv')
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20151 entries, 0 to 20150
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      20149 non-null  object
 1   FIRST_NAME                     20151 non-null  object
 2   PAY_RATE_TYPE                  20151 non-null  object
 3   PAY_RATE                       20151 non-null  int64 
 4   TITLE_DESCRIPTION              20151 non-null  object
 5   HOME_ORGANIZATION              20151 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  20151 non-null  object
 7   ORGANIZATION_LEVEL             20151 non-null  object
 8   TYPE_OF_REPRESENTATION         20151 non-null  object
 9   GENDER                         20151 non-null  object
 10  RUN_DATE                       20151 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.7+ MB


In [8]:
df_merged_2016 = pd.merge(df_2015, df_2016, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer', suffixes=('_2015', '_2016'))

In [9]:
df_merged_2017 = pd.merge(df_merged_2016, df_2017, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer', suffixes=(' ', '_2017'))

In [10]:
df_merged_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29015 entries, 0 to 29014
Data columns (total 27 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           29014 non-null  object 
 1   FIRST_NAME                          29015 non-null  object 
 2   PAY_RATE_TYPE_2015                  19623 non-null  object 
 3   PAY_RATE_2015                       19623 non-null  float64
 4   TITLE_DESCRIPTION_2015              19623 non-null  object 
 5   HOME_ORGANIZATION                   29015 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2015  19623 non-null  object 
 7   ORGANIZATION_LEVEL_2015             19623 non-null  object 
 8   TYPE_OF_REPRESENTATION_2015         19623 non-null  object 
 9   GENDER_2015                         19623 non-null  object 
 10  RUN_DATE_2015                       19623 non-null  object 
 11  PAY_RATE_TYPE_2016                  18144

In [11]:
df_merged_2018 = pd.merge(df_merged_2017, df_2018, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer', suffixes=('_2017', '_2018'))

In [12]:
df_merged_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34444 entries, 0 to 34443
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           34443 non-null  object 
 1   FIRST_NAME                          34444 non-null  object 
 2   PAY_RATE_TYPE_2015                  19960 non-null  object 
 3   PAY_RATE_2015                       19960 non-null  float64
 4   TITLE_DESCRIPTION_2015              19960 non-null  object 
 5   HOME_ORGANIZATION                   34444 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2015  19960 non-null  object 
 7   ORGANIZATION_LEVEL_2015             19960 non-null  object 
 8   TYPE_OF_REPRESENTATION_2015         19960 non-null  object 
 9   GENDER_2015                         19960 non-null  object 
 10  RUN_DATE_2015                       19960 non-null  object 
 11  PAY_RATE_TYPE_2016                  18489

In [13]:
df_merged_2019 = pd.merge(df_merged_2018, df_2019, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer')

In [14]:
df_merged_2019['TITLE_DESCRIPTION_2015'].value_counts()

TEACHER,FULL TIME                 6709
TEACHER,SPEC EDUCATION            1335
NOON TIME AIDE, 3 HRS              850
NOON TIME AIDE, 4 HRS              826
ONE TO ONE ASST, SPECIAL ED        691
                                  ... 
COORD,FOOD SERVICES,FULL SVCS        1
DIR,PAYROLL                          1
DEPUTY,EMPLOYEE SUPPORT SRVS         1
CHIEF OPERATING OFFICER              1
DEPUTY,PARENT COORD & SERVICES       1
Name: TITLE_DESCRIPTION_2015, Length: 541, dtype: int64

In [15]:
df_merged_2019.loc[(df_merged_2019['PAY_RATE_2015'].isna())&(df_merged_2019['TITLE_DESCRIPTION_2016']=='TEACHER,FULL TIME')]

,LAST_NAME,FIRST_NAME,PAY_RATE_TYPE_2015,PAY_RATE_2015,TITLE_DESCRIPTION_2015,HOME_ORGANIZATION,HOME_ORGANIZATION_DESCRIPTION_2015,ORGANIZATION_LEVEL_2015,TYPE_OF_REPRESENTATION_2015,GENDER_2015,...,GENDER_2018,RUN_DATE_2018,PAY_RATE_TYPE,PAY_RATE,TITLE_DESCRIPTION,HOME_ORGANIZATION_DESCRIPTION,ORGANIZATION_LEVEL,TYPE_OF_REPRESENTATION,GENDER,RUN_DATE
20523,ABDALLAH,JUWAYRIYAH,NaN,NaN,NaN,1470,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20532,ADAIR,DARRELL,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,...,M,4/1/2018,SALARIED,67706.0,"TEACHER,FULL TIME","FRANKLIN, BENJAMIN HIGH SCHOOL",HIGH SCHOOL,PFT-TEACHER,M,4/1/2019
20533,ADAMS,CA-TISHA,NaN,NaN,NaN,4394,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20536,ADAMS,KATHY,NaN,NaN,NaN,7220,NaN,NaN,NaN,NaN,...,F,4/1/2018,SALARIED,76461.0,"TEACHER,FULL TIME","CARNELL, LAURA H. SCHOOL",ELEMENTARY SCHOOL,PFT-TEACHER,F,4/1/2019
20537,ADAMS,LYNDA,NaN,NaN,NaN,8410,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24560,ZIEVE,HANNAH,NaN,NaN,NaN,6460,NaN,NaN,NaN,NaN,...,F,4/1/2018,SALARIED,54364.0,"TEACHER,FULL TIME",HILL-FREEDMAN WORLD ACADEMY,HIGH SCHOOL,PFT-TEACHER,F,4/1/2019
24561,ZIMMER,ASHLEY,NaN,NaN,NaN,2210,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24563,ZOTTER,JORDAN,NaN,NaN,NaN,5320,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24567,ZWOLAK,PAUL,NaN,NaN,NaN,2290,NaN,NaN,NaN,NaN,...,M,4/1/2018,SALARIED,69207.0,"TEACHER,FULL TIME",FRANKLIN LEARNING CENTER,HIGH SCHOOL,PFT-TEACHER,M,4/1/2019


In [16]:
school_df_2018 = pd.read_excel('data/SPR_SY1718_School_Metric_Scores_20190129.xlsx', sheet_name='SPR SY2017-2018')
school_df_2018.sample(10)

,School,SRC School ID,ULCS Code,Report,Rpt Type Long,Street Address,City,State,Zip Code,Phone Number,...,FAFSA Tier,Student Survey College & Career Score,Student Survey College & Career Pts Earn,Student Survey College & Career Pts Poss,Student Survey College & Career Pct Earn,Student Survey College & Career Tier,Teach Effect Distinguished Score,Teach Effect Instruction Score,Teacher Attendance Score,Student Survey Teaching Score
39,Franklin Learning Center,229,2290,HS,High School,616 N. 15th St.,Philadelphia,PA,19130,215-400-7570,...,REINFORCE,36,0.36,1,36,WATCH,Data Not Available,Data Not Available,52,46
185,William Rowen School,753,7530,ES,Elementary School,6841 N. 19th St.,Philadelphia,PA,19126,215-400-3550,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,60,0
193,Austin Meehan School,814,8140,MS,Middle School,3001 Ryan Ave.,Philadelphia,PA,19152,215-400-3240,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,78,34
313,Universal Charter School at Alcorn,X24,3424,K8,K8 School,3200 Dickinson St.,Philadelphia,PA,19146,215-952-6219,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,Data Not Available,81
137,Charles W. Henry School,625,6250,K8,K8 School,601 Carpenter Ln.,Philadelphia,PA,19119,215-400-3480,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,60,58
223,Imhotep Institute Charter High School,W09,3309,HS,High School,6201 N. 21st Street,Philadelphia,PA,19138,215-438-4140,...,MODEL,36,0.36,1,36,WATCH,Data Not Available,Data Not Available,Data Not Available,37
12,Andrew Hamilton School,129,1290,K8,K8 School,5640 Spruce St.,Philadelphia,PA,19139,215-400-7650,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,46,0
269,KIPP Philadelphia Charter School,W70,3370,K8,K8 School,2539 N. 16th St.,Philadelphia,PA,19132,267-758-2630,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,Data Not Available,42
67,Murrell Dobbins Career and Technical High School,406,4060,HS,High School,2150 W. Lehigh Ave.,Philadelphia,PA,19132,215-400-7050,...,MODEL,37,0.37,1,37,WATCH,Data Not Available,Data Not Available,71,42
147,Edward T. Steel School,639,6390,K8,K8 School,4301 Wayne Ave.,Philadelphia,PA,19140,215-400-3910,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,39,54


In [17]:
school_col_keep = ['School', 'ULCS Code', 'Report', 'Turnaround Model', 'Enrollment',
                   'Grades Served', 'Admissions Type', 'Overall Score', 'Overall Tier', 
                   'Ach Score', 'Ach Tier', 'Prog Score', 'Prog Tier', 'Clim Score', 
                   'Clim Tier', 'Attendance (95%+) Score', 'Attendance (95%+) Tier', 
                   'Retention Score', 'Retention Tier', 'ISS Score', 'ISS Tier', 'OSS Score',
                   'OSS Tier', 'Student Survey Climate Score', 'Student Survey Climate Tier', 
                   'Parent Survey Climate Score', 'Parent Survey Climate Tier', 
                   'Parent Survey Participation Score', 'Parent Survey Participation Tier', 
                   'Teacher Attendance Score','Student Survey Teaching Score']
school_df_2018=school_df_2018[school_col_keep]

In [18]:
school_df_2018.head()

,School,ULCS Code,Report,Turnaround Model,Enrollment,Grades Served,Admissions Type,Overall Score,Overall Tier,Ach Score,...,OSS Score,OSS Tier,Student Survey Climate Score,Student Survey Climate Tier,Parent Survey Climate Score,Parent Survey Climate Tier,Parent Survey Participation Score,Parent Survey Participation Tier,Teacher Attendance Score,Student Survey Teaching Score
0,John Bartram High School,1010,HS,NaN,561,'9-12',Neighborhood,13,INTERVENE,0,...,82,WATCH,52,REINFORCE,0,INTERVENE,2,INTERVENE,52,43
1,West Philadelphia High School,1020,HS,Turnaround,478,'9-12',Neighborhood,8,INTERVENE,0,...,76,INTERVENE,0,INTERVENE,0,INTERVENE,8,INTERVENE,49,0
2,High School of the Future,1030,HS,NaN,488,'9-12',Citywide,25,WATCH,0,...,87,REINFORCE,55,REINFORCE,0,INTERVENE,2,INTERVENE,56,36
3,Paul Robeson High School for Human Services,1050,HS,NaN,299,'9-12',Citywide,44,WATCH,3,...,95,MODEL,64,REINFORCE,82,MODEL,13,INTERVENE,68,53
4,William L. Sayre High School,1100,HS,NaN,425,'9-12',Neighborhood,8,INTERVENE,0,...,82,WATCH,0,INTERVENE,0,INTERVENE,8,INTERVENE,71,0


In [19]:
school_df_2018.loc[:,['Turnaround Model']]=school_df_2018['Turnaround Model'].fillna(value='None')

In [20]:
school_df_2018.isna().sum()

School                               0
ULCS Code                            0
Report                               0
Turnaround Model                     0
Enrollment                           0
Grades Served                        0
Admissions Type                      0
Overall Score                        0
Overall Tier                         0
Ach Score                            0
Ach Tier                             0
Prog Score                           0
Prog Tier                            0
Clim Score                           0
Clim Tier                            0
Attendance (95%+) Score              0
Attendance (95%+) Tier               0
Retention Score                      0
Retention Tier                       0
ISS Score                            0
ISS Tier                             0
OSS Score                            0
OSS Tier                             0
Student Survey Climate Score         0
Student Survey Climate Tier          0
Parent Survey Climate Sco

In [21]:
teacher_df_2018 = pd.read_csv('data/employee_information_April2018.csv')

In [22]:
teacher_df_2019 = pd.read_csv('data/employee_information_April2019.csv')

In [23]:
merged_df= pd.merge(teacher_df_2018, teacher_df_2019, 
                    on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                    how='outer', suffixes=('_2018', '_2019'))

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25275 entries, 0 to 25274
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           25273 non-null  object 
 1   FIRST_NAME                          25275 non-null  object 
 2   PAY_RATE_TYPE_2018                  19279 non-null  object 
 3   PAY_RATE_2018                       19279 non-null  float64
 4   TITLE_DESCRIPTION_2018              19279 non-null  object 
 5   HOME_ORGANIZATION                   25275 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2018  19279 non-null  object 
 7   ORGANIZATION_LEVEL_2018             19279 non-null  object 
 8   TYPE_OF_REPRESENTATION_2018         18962 non-null  object 
 9   GENDER_2018                         19279 non-null  object 
 10  RUN_DATE_2018                       19279 non-null  object 
 11  PAY_RATE_TYPE_2019                  20254

In [25]:
merged_df['TITLE_DESCRIPTION_2018'].value_counts()

TEACHER,FULL TIME                6789
SPECIAL EDUCATION ASSISTANT      1848
TEACHER,SPEC EDUCATION           1389
GENERAL CLEANER, 8 HOURS          558
FOOD SVCS ASSISTANT               428
                                 ... 
REAL PROPERTY MGMT SPECIALIST       1
MANAGER,DISTRICT PERFORMANCE        1
DISCIPLINARY HEARING OFFICER        1
DEPUTY CHIEF,EARLY CHILDHOOD        1
RET PRIN,BUILDING ADM,LTS           1
Name: TITLE_DESCRIPTION_2018, Length: 591, dtype: int64

In [26]:
retention_2019 = merged_df.loc[(merged_df['TITLE_DESCRIPTION_2018']=='TEACHER,FULL TIME')|
                               (merged_df['TITLE_DESCRIPTION_2018']=='TEACHER,SPEC EDUCATION')]


In [27]:
retention_2019.isna().sum()

LAST_NAME                                1
FIRST_NAME                               0
PAY_RATE_TYPE_2018                       0
PAY_RATE_2018                            0
TITLE_DESCRIPTION_2018                   0
HOME_ORGANIZATION                        0
HOME_ORGANIZATION_DESCRIPTION_2018       0
ORGANIZATION_LEVEL_2018                  0
TYPE_OF_REPRESENTATION_2018              0
GENDER_2018                              0
RUN_DATE_2018                            0
PAY_RATE_TYPE_2019                    1593
PAY_RATE_2019                         1593
TITLE_DESCRIPTION_2019                1593
HOME_ORGANIZATION_DESCRIPTION_2019    1593
ORGANIZATION_LEVEL_2019               1593
TYPE_OF_REPRESENTATION_2019           1593
GENDER_2019                           1593
RUN_DATE_2019                         1593
dtype: int64

In [28]:
school_df_2018.head()

,School,ULCS Code,Report,Turnaround Model,Enrollment,Grades Served,Admissions Type,Overall Score,Overall Tier,Ach Score,...,OSS Score,OSS Tier,Student Survey Climate Score,Student Survey Climate Tier,Parent Survey Climate Score,Parent Survey Climate Tier,Parent Survey Participation Score,Parent Survey Participation Tier,Teacher Attendance Score,Student Survey Teaching Score
0,John Bartram High School,1010,HS,None,561,'9-12',Neighborhood,13,INTERVENE,0,...,82,WATCH,52,REINFORCE,0,INTERVENE,2,INTERVENE,52,43
1,West Philadelphia High School,1020,HS,Turnaround,478,'9-12',Neighborhood,8,INTERVENE,0,...,76,INTERVENE,0,INTERVENE,0,INTERVENE,8,INTERVENE,49,0
2,High School of the Future,1030,HS,None,488,'9-12',Citywide,25,WATCH,0,...,87,REINFORCE,55,REINFORCE,0,INTERVENE,2,INTERVENE,56,36
3,Paul Robeson High School for Human Services,1050,HS,None,299,'9-12',Citywide,44,WATCH,3,...,95,MODEL,64,REINFORCE,82,MODEL,13,INTERVENE,68,53
4,William L. Sayre High School,1100,HS,None,425,'9-12',Neighborhood,8,INTERVENE,0,...,82,WATCH,0,INTERVENE,0,INTERVENE,8,INTERVENE,71,0


In [29]:
retention_2019.loc[:,['HOME_ORGANIZATION']] = pd.to_numeric(retention_2019['HOME_ORGANIZATION'], 
                                                    errors='coerce')

/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [30]:
retention_2019.isna().sum()

LAST_NAME                                1
FIRST_NAME                               0
PAY_RATE_TYPE_2018                       0
PAY_RATE_2018                            0
TITLE_DESCRIPTION_2018                   0
HOME_ORGANIZATION                       93
HOME_ORGANIZATION_DESCRIPTION_2018       0
ORGANIZATION_LEVEL_2018                  0
TYPE_OF_REPRESENTATION_2018              0
GENDER_2018                              0
RUN_DATE_2018                            0
PAY_RATE_TYPE_2019                    1593
PAY_RATE_2019                         1593
TITLE_DESCRIPTION_2019                1593
HOME_ORGANIZATION_DESCRIPTION_2019    1593
ORGANIZATION_LEVEL_2019               1593
TYPE_OF_REPRESENTATION_2019           1593
GENDER_2019                           1593
RUN_DATE_2019                         1593
dtype: int64

Drop the null values from the `HOME ORGANIZATION`. This is the column we will use to merge with the school data and it has to be numerical. The staff with the non-numeric codes are assigned to locations that are not actual schools such as code 9KT0 represents the Office of High School Support.

In [35]:
retention_2019.dropna(subset = ['HOME_ORGANIZATION'], inplace=True)

<ipython-input-35-220873fa0ef4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_2019.dropna(subset = ['HOME_ORGANIZATION'], inplace=True)


In [36]:
retention_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8085 entries, 4 to 19277
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           8084 non-null   object 
 1   FIRST_NAME                          8085 non-null   object 
 2   PAY_RATE_TYPE_2018                  8085 non-null   object 
 3   PAY_RATE_2018                       8085 non-null   float64
 4   TITLE_DESCRIPTION_2018              8085 non-null   object 
 5   HOME_ORGANIZATION                   8085 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  8085 non-null   object 
 7   ORGANIZATION_LEVEL_2018             8085 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         8085 non-null   object 
 9   GENDER_2018                         8085 non-null   object 
 10  RUN_DATE_2018                       8085 non-null   object 
 11  PAY_RATE_TYPE_2019                  6521 n

In [37]:
# ULCS Code
full_df= pd.merge(retention_2019, school_df_2018, 
                  left_on='HOME_ORGANIZATION', right_on='ULCS Code',
                  how='left')

In [38]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8252 entries, 0 to 8251
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           8251 non-null   object 
 1   FIRST_NAME                          8252 non-null   object 
 2   PAY_RATE_TYPE_2018                  8252 non-null   object 
 3   PAY_RATE_2018                       8252 non-null   float64
 4   TITLE_DESCRIPTION_2018              8252 non-null   object 
 5   HOME_ORGANIZATION                   8252 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  8252 non-null   object 
 7   ORGANIZATION_LEVEL_2018             8252 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         8252 non-null   object 
 9   GENDER_2018                         8252 non-null   object 
 10  RUN_DATE_2018                       8252 non-null   object 
 11  PAY_RATE_TYPE_2019                  6663 no

In [41]:
full_df[full_df.filter(like='School').isnull().any(1)]

,LAST_NAME,FIRST_NAME,PAY_RATE_TYPE_2018,PAY_RATE_2018,TITLE_DESCRIPTION_2018,HOME_ORGANIZATION,HOME_ORGANIZATION_DESCRIPTION_2018,ORGANIZATION_LEVEL_2018,TYPE_OF_REPRESENTATION_2018,GENDER_2018,...,OSS Score,OSS Tier,Student Survey Climate Score,Student Survey Climate Tier,Parent Survey Climate Score,Parent Survey Climate Tier,Parent Survey Participation Score,Parent Survey Participation Tier,Teacher Attendance Score,Student Survey Teaching Score
14,ABRAHAM-CUFF,NAFHRAH,SALARIED,48527.0,"TEACHER,SPEC EDUCATION",6400.0,WIDENER MEMORIAL SCHOOL,ELEMENTARY SCHOOL,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,ABT,DEBBIE,SALARIED,78376.0,"TEACHER,FULL TIME",6392.0,STEEL SCHOOL HEAD START,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,ALLEN,HEATHER,SALARIED,52196.0,"TEACHER,FULL TIME",6202.0,"DAY, ANNA B. HEAD START",EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,ALLEN,MAYA,SALARIED,45359.0,"TEACHER,FULL TIME",6341.0,PENNELL HEAD START,EARLY CHILDHOOD,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,ALLMAN,ELIZABETH,SALARIED,59532.0,"TEACHER,FULL TIME",4399.0,WRIGHT HEAD START,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8167,YORKONIS,TRACY,SALARIED,67789.0,"TEACHER,FULL TIME",1475.0,HAVERFORD CTR BRIGHT FUTURES,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8185,YU,JUYEON,SALARIED,90051.0,"TEACHER,FULL TIME",6400.0,WIDENER MEMORIAL SCHOOL,ELEMENTARY SCHOOL,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8197,ZAJDEL,HEATHER,SALARIED,90051.0,"TEACHER,FULL TIME",8460.0,PHILA LEARNING ACADEMY-SOUTH,TRANSITION / OVERAGE SCHOOL,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8213,ZELNER,KATHLEEN,SALARIED,69623.0,"TEACHER,FULL TIME",8272.0,HOLME HEAD START,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After the merge, we can see that the teachers with no school listed are those from primarily Pre-K or Transition/Overage Schools where we don't have School Progress Report data for those schools. We will drop those ~250 rows to focus on teachers in Philadelphia schools participated in the 2017-2018 School Progress Report.

In [42]:
full_df.dropna(subset = ['School'], inplace=True)

In [43]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7984 entries, 0 to 8251
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           7983 non-null   object 
 1   FIRST_NAME                          7984 non-null   object 
 2   PAY_RATE_TYPE_2018                  7984 non-null   object 
 3   PAY_RATE_2018                       7984 non-null   float64
 4   TITLE_DESCRIPTION_2018              7984 non-null   object 
 5   HOME_ORGANIZATION                   7984 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  7984 non-null   object 
 7   ORGANIZATION_LEVEL_2018             7984 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         7984 non-null   object 
 9   GENDER_2018                         7984 non-null   object 
 10  RUN_DATE_2018                       7984 non-null   object 
 11  PAY_RATE_TYPE_2019                  6450 no

We will establish a `TURNOVER` column that will act as the target column for modeling purposes. To do this, if a record has non-null values from the 2019 dataset, then they will be marked as 0 indicating they were retained in their 2018 school. However, if a record has a null value in the 2019 dataset, they will be marked with a 1 indicating they turned over from their 2018 school.

In [44]:
full_df['TURNOVER']=np.where(full_df['PAY_RATE_2019'].isnull(), 1,0)

In [47]:
full_df['TURNOVER'].value_counts(normalize=True)

0    0.807866
1    0.192134
Name: TURNOVER, dtype: float64

From this, we can see that nearly 20% of the public school teachers left their schools after the 2018 school year. This does indicate that there is a class imbalance at play that will need to be accounted for when modeling.

We still need to clean up some of the data before modeling. We can drop columns that will not be helpful for modeling, such as the teacher data from 2019 since it contains null values from those teachers that left their school.

### Modeling

### Evaluation